In [11]:
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local-advanced.ipynb

from azureml.core import  Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

from azureml.core.environment import Environment 
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model,InferenceConfig

# interactive_auth = InteractiveLoginAuthentication(tenant_id="9ce70869-60db-44fd-abe8-d2767077fc8f")
ws = Workspace.from_config()


In [12]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: cselscdhazureml
Azure region: eastus2
Subscription id: 320d8d57-c87c-4434-827f-59ee7d86687a
Resource group: CSELS-CDH-DEV


In [13]:
import sklearn
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

print(sklearn.__version__)

1.1.2


In [14]:
model = Model(workspace=ws,name='breast-cancer',version=8)

In [15]:
import os
print(os.getenv("AZUREML_MODEL_DIR"))
#model_path = Model.get_model_path(os.getenv("AZUREML_MODEL_DIR").split('/')[-2])

None


In [17]:
import os

# os.makedirs('./aml_outputs/',exist_ok=True)
# model.download(target_dir='./aml_outputs/',exist_ok=True)

In [18]:
%%writefile ./scoring/score.py

import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION)
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'cancer_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)



input_sample = np.array([[0.393845668409139, -0.7457496952627328, -0.22189398515428446, -0.7676382304726264, -0.6999246324662508, -0.17745016145311038, 
-0.81484548808207, -0.770581761525873, -0.7189314051409194, 0.07367558407930598, -0.4665409990624931, -0.06425072325003647, -0.667435753669589, 
-0.10209930505592255, -0.2928749021935234, -0.18393920631817678, -0.8204798568998558, -0.6065568523529427, -0.395651155530143, 0.3134950272756633,
 -0.8678658791041564, -0.6897420117050609, -0.5598110994362666, -0.723009965205552, -0.6542518616646612, -0.5936861839456196,
 -0.9574791012332506, -0.9270503937797329, -0.7552651134179409, -0.2651797994882257, -1.0640128515011344]])
output_sample = np.array([0])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        print("input_data....")
        print(type(data))
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return "here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

Overwriting ./scoring/score.py


### use custom envirronment

In [19]:
print('scikit-learn=={}'.format(sklearn.__version__))

scikit-learn==1.1.2


In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment.from_conda_specification(name='mlopspython_scoring', file_path='./conda_dependencies_scoring.yml')
environment.docker.enabled = True
#environment = Environment('breast_cancer_scoring_env')
#environment.python.conda_dependencies = CondaDependencies.create(
#    python_version='3.8',
#    conda_packages=[
#    'pip==20.2.4'],
#    pip_packages=[
#    'azureml-defaults',
#    'pandas',
#    'inference-schema[numpy-support]',
#    'joblib',
#    'numpy',
#    'scikit-learn'
#])

#### Create Inference Configuration

* file_path: input parameter to Environment constructor. Manages conda and python package dependencies.
* env.docker.base_dockerfile: any extra steps you want to inject into docker file
* source_directory: holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
* entry_script: contains logic specific to initializing your model and running predictions

In [22]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'breast-cancer-custom-service'

inference_config = InferenceConfig(entry_script='./scoring/score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                memory_gb=1 )
                                                #vnet_name='csels-cdh-dev-vnet', 
                                                #subnet_name='cdh-azml-dev-snet')



service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-09-08 09:16:20-04:00 Creating Container Registry if not exists.
2022-09-08 09:16:20-04:00 Registering the environment.
2022-09-08 09:16:21-04:00 Building image..
2022-09-08 09:26:35-04:00 Generating deployment configuration.
2022-09-08 09:26:37-04:00 Submitting deployment to compute..
2022-09-08 09:26:40-04:00 Checking the status of deployment breast-cancer-custom-service..
2022-09-08 09:28:21-04:00 Checking the status of inference endpoint breast-cancer-custom-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
# print(service.get_logs())

In [23]:
import json
import numpy as np
import pandas as pd 

input_sample = np.array([[0.393845668409139, -0.7457496952627328, -0.22189398515428446, -0.7676382304726264, -0.6999246324662508, -0.17745016145311038, 
-0.81484548808207, -0.770581761525873, -0.7189314051409194, 0.07367558407930598, -0.4665409990624931, -0.06425072325003647, -0.667435753669589, 
-0.10209930505592255, -0.2928749021935234, -0.18393920631817678, -0.8204798568998558, -0.6065568523529427, -0.395651155530143, 0.3134950272756633,
 -0.8678658791041564, -0.6897420117050609, -0.5598110994362666, -0.723009965205552, -0.6542518616646612, -0.5936861839456196,
 -0.9574791012332506, -0.9270503937797329, -0.7552651134179409, -0.2651797994882257, -1.0640128515011344]])

input_payload = json.dumps({
    'data': input_sample.tolist()
})

output = service.run(input_payload)

print(output)

here is your result = [0.]


## AKS deployment

In [26]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-secure-workspace-vnet?tabs=pe%2Ccli

from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

service_name = 'breast-cancer-custom-service-aks'

# TODO Check if compute exits
aks_target = AksCompute(ws,"aks-cdh-dev")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1)
aks_service = Model.deploy(ws, 
                        service_name, 
                        [model], 
                        inference_config, 
                        deployment_config, 
                        aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)
print(aks_service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-09-08 09:55:27-04:00 Creating Container Registry if not exists.
2022-09-08 09:55:27-04:00 Registering the environment.
2022-09-08 09:55:28-04:00 Use the existing image.
2022-09-08 09:55:29-04:00 Creating resources in AKS.
2022-09-08 09:55:30-04:00 Submitting deployment to compute.
2022-09-08 09:55:31-04:00 Checking the status of deployment breast-cancer-custom-service-aks..
2022-09-08 09:57:20-04:00 Checking the status of inference endpoint breast-cancer-custom-service-aks.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
/bin/bash: /azureml-envs/azureml_07aaf71f4332d71d1897c2e84e139941/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_07aaf71f4332d71d1897c2e84e139941/lib/libtinfo.so.6: no version informati

In [27]:
import json
import numpy as np
import pandas as pd 

input_sample = np.array([[0.393845668409139, -0.7457496952627328, -0.22189398515428446, -0.7676382304726264, -0.6999246324662508, -0.17745016145311038, 
-0.81484548808207, -0.770581761525873, -0.7189314051409194, 0.07367558407930598, -0.4665409990624931, -0.06425072325003647, -0.667435753669589, 
-0.10209930505592255, -0.2928749021935234, -0.18393920631817678, -0.8204798568998558, -0.6065568523529427, -0.395651155530143, 0.3134950272756633,
 -0.8678658791041564, -0.6897420117050609, -0.5598110994362666, -0.723009965205552, -0.6542518616646612, -0.5936861839456196,
 -0.9574791012332506, -0.9270503937797329, -0.7552651134179409, -0.2651797994882257, -1.0640128515011344]])

input_payload = json.dumps({
    'data': input_sample.tolist()
})

output = aks_service.run(input_payload)

print(output)

here is your result = [0.]
